<div style="position:relative; height:160px;">
<img src="logoDECATUFS.png" width="500" height="500" style="position:absolute; left:-8px;">
</div>

<div style="text-align:center;font-size:17px; white-space:pre;margin-top:-35px">
        <h3>             Modelos Lineares Generalizados</h3>
        Disciplina ofertada pelo DECAT/UFS
        Código: ESTAT0092
        Nível: Graduação
        Carga horária: 60h
        Período: 2020.2
        Professor responsável e ministrante: Luiz Henrique Dore
        <h3>            Aula 12: testanto a significância global do modelo</h3>
</div>

# Exemplo 3

<p style="text-align: justify; font-size:16px">
Considere o conjunto de dados <b>trees.dat</b>, descrito no exemplo 4 da aula 2. O objetivo é ajustar um modelo de regressão gama, com ligação logarítmica, no qual a variável resposta é o volume e as variáveis preditoras são a altura e o diâmetro e executar o teste da razão de verossimilhanças para avaliar a significância global do modelo.
<p>

### 3.1 Leitura e visualização dos dados

In [1]:
dados<-read.table("/Dados/Google_Drive_UFS/Aulas/2020_2/MLG/Dados/trees.dat",header=TRUE)
n<-dim(dados)[1] #Número de observações
head(dados) #Visualizando as primeiras observações

,Diametro,Altura,Volume
,<dbl>,<int>,<dbl>
1,8.3,70,10.3
2,8.6,65,10.3
3,8.8,63,10.2
4,10.5,72,16.4
5,10.7,81,18.8
6,10.8,83,19.7


In [2]:
diam<-dados[,1]
alt<-dados[,2]
vol<-dados[,3]

### 3.2 Formulação do modelo

<p style="text-align: justify; font-size:15px">
    <ol style="font-size:16px">
        <li>$Volume\sim Gama(a,b)$;</li>
        <li>$\eta=\beta_0+\beta_1\cdot Altura+\beta_2\cdot Diâmetro$;</li>
        <li>$\ln\left(\mu\right)=\eta$.</li>
    </ol>
</p>

### 3.3 Estimação dos parâmetros pelo método escore de Fisher usando as funções ``glm`` e ``gamma.shape``.

<p style="text-align: justify;font-size: 16px">Carregando os pacotes necessários.</p>

In [3]:
library(MASS) #Contém a função gamma.shape

<p style="text-align: justify;font-size: 16px">Estimação dos parâmetros no modelo completo.</p>

In [4]:
aj<-glm(formula=Volume~Diametro+Altura,family=Gamma(link="log"),data=dados)
beta<-aj$coefficients

In [5]:
ajPhi<-gamma.shape(aj)
alpha<-ajPhi$alpha
phi<-1/alpha
names(phi)<-"Dispersão"

<p style="text-align: justify;font-size: 16px">Visualizando as estimativas dos parâmetros no modelo completo.</p>

In [6]:
print(c(beta,phi))

(Intercept)    Diametro      Altura   Dispersão 
0.092304581 0.145281192 0.016577883 0.008455011 


<p style="text-align: justify;font-size: 16px">Estimação dos coeficientes de regressão no modelo sob $H_0$. No argumento <font face="'Courier New'">formula</font>, digite apenas o número 1 na parte onde se informa as variáveis preditoras.</p>

In [7]:
aj0<-glm(formula=Volume~1,family=Gamma(link="log"),data=dados)
beta0<-aj0$coefficients

<p style="text-align: justify;font-size: 16px">Estimação do parâmetro de dispersão no modelo sob $H_0$.</p>

In [8]:
aj0Phi<-gamma.shape(aj0)
alpha0<-aj0Phi$alpha
phi0<-1/alpha0
names(phi0)<-"Dispersão"

<p style="text-align: justify;font-size: 16px">Visualizando as estimativas dos parâmetros no sob $H_0$.</p>

In [9]:
print(c(beta0,phi0))

(Intercept)   Dispersão 
  3.4068801   0.2573312 


### 3.4 Executando o teste da razão de verossimilhanças

<p style="text-align: justify; font-size:16px">No caso da distribuição gama, a função de log-verossimilhança é dada por (ver a fórmula 2 no tópico 10 e o exemplo 4 no tópico 12)
    $$l(\boldsymbol{\beta},\phi)=\phi^{-1}\sum\limits_{i=1}^n[\theta_iy_i-b(\theta_i)]+\sum\limits_{i=1}^nc(y_i,\phi),$$
onde $\theta_i=-\frac{1}{\mu_i}$, $b(\theta_i)=-\ln(-\theta_i)$ e $c(y_i,\phi)=\phi^{-1}\ln(\phi^{-1})-\ln\Gamma(\phi^{-1})+(\phi^{-1}-1)\ln(y_i)$. Primeiramente, calcula-se os valores de $\mu_i$ utilizando a função <font face="'Courier New'">predict</font>. Utilizando os valores de $\mu_i$, calcula-se os valores de $\theta_i$ e, utilizando os valores de $\theta_i$, calcula-se os valores de $b(\theta_i)$. Por último, calcula-se os valores de $c(y_i,\phi)$ e substitui-se os valores de $\theta_i$, $b(\theta_i)$ e $c(y_i,\phi)$ na fórmula da função de log-verossimilhança.

<p style="text-align: justify;font-size: 16px">Calculando a log-verossimilhança do modelo completo.</p>

In [10]:
y<-vol #Atribuindo os valores observados da resposta ao vetor y
mu<-predict(aj,type="response") #Calculando os valores de mu com base no modelo ajustado
theta<--1/mu #Calculando os valores de theta a partir de mu
btheta<--log(-theta) #Calculando os valores de b(theta) a partir de theta
cyphi<-phi^(-1)*log(phi^(-1))-log(gamma(phi^(-1)))+(phi^(-1)-1)*log(y) #Calculando os valores de c(y,phi)
l<-phi^(-1)*sum(theta*y-btheta)+sum(cyphi) #Calculando a log-verossimilhança
l<-as.numeric(l)
print(paste("log-verossimilhança do modelo completo: ",l))

[1] "log-verossimilhança do modelo completo:  -71.5040244230622"


<p style="text-align: justify;font-size: 16px">Calculando a log-verossimilhança do modelo sob $H_0$.</p>

In [11]:
y<-vol #Atribuindo os valores observados da resposta ao vetor y
mu0<-predict(aj0,type="response") #Calculando os valores de mu com base no modelo ajustado
theta0<--1/mu0 #Calculando os valores de theta a partir de mu
btheta0<--log(-theta0) #Calculando os valores de b(theta) a partir de theta
cyphi0<-phi0^(-1)*log(phi0^(-1))-log(gamma(phi0^(-1)))+(phi0^(-1)-1)*log(y) #Calculando os valores de c(y,phi)
l0<-phi0^(-1)*sum(theta0*y-btheta0)+sum(cyphi0) #Calculando a log-verossimilhança
l0<-as.numeric(l0)
print(paste("log-verossimilhança do modelo sob H0: ",l0))

[1] "log-verossimilhança do modelo sob H0:  -125.726043079491"


<p style="text-align: justify;font-size: 16px">Calculando a estatística do teste de da razão de verossimilhanças $T_{RV}=2\left\{l\left(\boldsymbol{\widehat\beta},\hat\phi\right)-l\left(\hat\beta^0_0,{\hat\phi}^0\right)\right\}$.</p>

In [12]:
tRV<-2*(l-l0)
print(paste("Valor observado da estatística do teste: ",tRV))

[1] "Valor observado da estatística do teste:  108.444037312857"


<p style="text-align: justify;font-size: 16px">Calculando o p-valor do teste. Sob $H_0$, a estatística $T_{RV}$ segue, aproximadamnete, distribuição $\chi_p^2$, onde $p$ é o número de variáveis preditoras. Logo, o p-valor fica dado por $P(T_{RV}>t_{RV})=1-P(T_{RV}\leq t_{RV})$, onde $T_{RV}\sim\chi_p^2$ e $t_{RV}$ é o valor observado de $T_{RV}$.</p>

In [13]:
p<-length(beta)-1 #O número de variáveis preditoras é o número de coeficientes menos 1 (excluindo o intercepto)
pvalor<-1-pchisq(tRV,p) 
print(paste("p-valor do teste: ",pvalor))

[1] "p-valor do teste:  0"


### 3.5 Executando o teste da razão de verossimilhanças usando a função ``lr.test``

<p style="text-align: justify;font-size: 16px">Carregando os pacotes necessários.</p>

In [14]:
library(mdscore) #Contém a função lr.test
#install.packages("mdscore") #Execute esse comando para instalar o pacote mdscore, caso já não o tenha instalado

<p style="text-align: justify;font-size: 16px">A função <font face="'Courier New'">lr.test</font> realiza o teste da razão de verossimilhanças para a significância global do modelo. A função recebe dois argumentos:</p>
    <ol style="text-align: justify; font-size:16px">
        <li><font face="'Courier New'">fit1</font>: modelo ajustado sob $H_0$. Saída da função <font face="'Courier New'">glm</font> referente ao modelo sob $H_0$.</li>
        <li><font face="'Courier New'">fit2</font>: modelo completo ajustado. Saída da função <font face="'Courier New'">glm</font> referente ao modelo completo.</li>
    </ol>
</p>

In [15]:
lr.test(aj0,aj)

$LR
[1] 108.4729

$pvalue
[1] 2.78873e-24

attr(,"class")
[1] "lrt.test"

<p style="text-align: justify; font-size:16px">
O p-valor é muito pequeno, indicando a rejeição de $H_0$. Ou seja, o resultado do teste indica que há uma associação estatisticamente significativa entre o volume e ao menos uma das duas variáveis preditoras.
</p>